In [7]:
import operator
import math
import random
import numpy
from functools import partial
from deap import algorithms, base, creator, tools, gp

# Define new functions
#funcion que cuida que no se divida entre 0
# Define a safe division function
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

#un main para asignar cuantas variables vamos a ocupar, en este caso 2
#definir operadores posibes
#suma con aridad 2
#resta con aridad 2
#multiplicacion con aridad 2
#division con aridad 2
#negacion con aridad 1
#sino con aridad 1
#coseno con aridad 1
# Create the primitive set with 2 inputs
pset = gp.PrimitiveSet("MAIN", 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand101", partial(random.randint, -1, 1))

# Rename arguments to x and y
#cambio para deje utilizar las variables como x y y enlugar de arg1 y arg 2
pset.renameArguments(ARG0='x', ARG1='y')

# Create the fitness and individual types
#creator permite crear nuevos tipos  de dato como infdividuo que representa a individuos en el algoritmo
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

#toolbox es de lo mas importante de deap  permite crear individuals, populations, and compiling the tree expressions into callable functions.
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

#en esta funcion es donde incertamos la funcion con la cual queremos evaluar en este caso se va a cambiar a :
#x^3*5y^2+x/2
#va a calcular el error entre los resultados y la funcion real.
def evalSymbReg(individual, points):
    func = toolbox.compile(expr=individual)
    # Calculate squared errors for each (x, y) point
    sqerrors = ((func(x, y) - (5 * x**3 * y**2 + x / 2))**2 for x, y in points)
    return math.fsum(sqerrors) / len(points),  # Return as a tuple
#ahora con toolbox se crean los operadores de evaluacion y los operadores geneticos.
# Generate points for evaluation, creating pairs of (x, y)
points = [(x/10.0, y/10.0) for x in range(-10, 11) for y in range(-10, 11)]
toolbox.register("evaluate", evalSymbReg, points=points)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)
    
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    # Run the genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    return pop, log, hof

if __name__ == "__main__":
    main()



   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max    	min     	nevals	std    	avg 	gen	max	min	nevals	std    
0  	300   	3.38812	0  	56.1632	0.634517	300   	4.81963	3.61	0  	7  	2  	300   	1.43686
1  	173   	2.02732	1  	19.5928	0.634517	173   	1.50977	3.71	1  	12 	2  	173   	1.75857
2  	167   	1.59255	2  	5.67047	0.633675	167   	0.784622	3.76667	2  	8  	1  	167   	1.60381
3  	182   	1.29077	3  	5.39593	0.633675	182   	0.693038	3.98   	3  	10 	1  	182   	1.87428
4  	154   	1.05407	4  	7.12998	0.633675	154   	0.661405	4.11   	4  	11 	1  	154   	1.96585
5  	167   	1.00822	5  	4.28845	0.633675	167   	0.610671	4.38   	5  	14 	1  	167   	1.99723
6  	171   	0.906072	6  	7.12998	0.633675	171   	0.58922 	4.45667	6  	12 	1  	171   	1.81516
7  	176   	4.32531 	7  	1036.33	0.633675	176   	59.68